In [38]:
import pandas as pd
import numpy as np

In [39]:
df = pd.read_csv("./Data/HINDALCO-I.NFO.csv").set_index("Time")
df = df.drop(columns=["Date"])
df.head()

,BID_PRICE,ASK_PRICE,LTQ,OpenInterest
Time,,,,
09:15:00.000,861.10,862.25,1400.0,62003200.0
09:15:01.000,861.85,862.90,15400.0,62003200.0
09:15:02.000,861.85,862.90,15400.0,62003200.0
09:15:03.000,862.30,863.90,23100.0,62003200.0
09:15:04.000,863.00,864.20,23100.0,62003200.0


In [40]:
df["MID_PRICE"] = (df["ASK_PRICE"] + df["BID_PRICE"])/2

In [41]:
print(df.head())

              BID_PRICE  ASK_PRICE      LTQ  OpenInterest  MID_PRICE
Time                                                                
09:15:00.000     861.10     862.25   1400.0    62003200.0    861.675
09:15:01.000     861.85     862.90  15400.0    62003200.0    862.375
09:15:02.000     861.85     862.90  15400.0    62003200.0    862.375
09:15:03.000     862.30     863.90  23100.0    62003200.0    863.100
09:15:04.000     863.00     864.20  23100.0    62003200.0    863.600


In [42]:
# Convert Time column to datetime index
# df.index = pd.to_datetime(df.index)
df.index = pd.to_datetime(df.index, format='%H:%M:%S.%f')
# Now resample to 1-minute intervals
df_1min = df['MID_PRICE'].resample('1min').last()

# Calculate log returns
log_returns = np.log(df_1min / df_1min.shift(1))

df["log_returns"] = log_returns

In [43]:


# Calculate log returns for every second
log_returns_1s = np.log(df['MID_PRICE'] / df['MID_PRICE'].shift(1))

# Remove the first NaN value
log_returns_1s = log_returns_1s.dropna()

df["seconds_log"] = log_returns_1s

In [44]:
df.to_csv("test.csv")

In [45]:

# Calculate 1-second log returns
log_returns_1s = np.log(df['MID_PRICE'] / df['MID_PRICE'].shift(1)).dropna()

# Create a DataFrame to store features
features_list = []

# Group by 1-minute intervals
for timestamp, group in log_returns_1s.resample('1min'):
    if len(group) > 0:
        # Convert second-level returns to a feature vector
        feature_dict = {'timestamp': timestamp}
        
        # Add each second's return as a feature (pad if < 60 seconds)
        returns_array = group.values
        for i in range(60):
            if i < len(returns_array):
                feature_dict[f'ret_sec_{i+1}'] = returns_array[i]
            else:
                feature_dict[f'ret_sec_{i+1}'] = 0.0  # or np.nan
        
        features_list.append(feature_dict)

# Convert to DataFrame
X = pd.DataFrame(features_list).set_index('timestamp')

# Calculate 1-minute log returns as target (shift to align correctly)
df_1min = df['MID_PRICE'].resample('1min').last()
y = np.log(df_1min / df_1min.shift(1)).dropna()

# Align features and target
X, y = X.align(y, join='inner', axis=0)

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
X

Features shape: (375, 60)
Target shape: (375,)


,ret_sec_1,ret_sec_2,ret_sec_3,ret_sec_4,ret_sec_5,ret_sec_6,ret_sec_7,ret_sec_8,ret_sec_9,ret_sec_10,...,ret_sec_51,ret_sec_52,ret_sec_53,ret_sec_54,ret_sec_55,ret_sec_56,ret_sec_57,ret_sec_58,ret_sec_59,ret_sec_60
1900-01-01 09:16:00,0.000029,-0.000029,-0.000145,0.000000,0.000116,0.000000,0.000290,-0.000058,0.000000,-0.000116,...,-0.000436,-0.000407,0.000000,-0.000087,0.000900,0.000029,-2.612975e-04,0.000348,0.000145,0.000145
1900-01-01 09:17:00,-0.000029,-0.000116,-0.000377,0.000435,-0.000232,-0.000232,-0.000116,-0.000029,0.000174,-0.000029,...,-0.000175,-0.000058,-0.000320,0.000117,0.000000,-0.000699,2.220446e-16,-0.000029,0.000117,0.000000
1900-01-01 09:18:00,0.000087,-0.000262,0.000029,0.000000,0.000408,0.000175,0.000087,0.000117,-0.000233,-0.000175,...,0.000087,-0.000175,-0.000029,0.000000,0.000000,-0.000029,5.827166e-05,-0.000029,-0.000029,0.000058
1900-01-01 09:19:00,-0.000117,0.000437,-0.000437,0.000029,0.000262,0.000058,-0.000117,0.000408,-0.000262,0.000029,...,0.000087,0.000029,0.000058,-0.000146,0.000029,0.000058,-8.735532e-05,-0.000175,-0.000029,-0.000117
1900-01-01 09:20:00,0.000000,-0.000146,0.000117,0.000000,-0.000029,0.000000,0.000000,0.000058,0.000175,-0.000146,...,0.000000,0.000029,0.000029,0.000000,-0.000029,-0.000029,-5.846757e-05,0.000088,-0.000585,-0.000322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1900-01-01 15:26:00,0.000000,-0.000029,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-0.000059,0.000029,0.000029,0.000000e+00,0.000000,0.000059,0.000059
1900-01-01 15:27:00,-0.000029,0.000000,0.000000,0.000000,0.000029,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
1900-01-01 15:28:00,0.000000,-0.000146,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000117,0.000000,-0.000029,0.000000,0.000000e+00,-0.000029,0.000000,0.000000
1900-01-01 15:29:00,0.000088,0.000000,0.000000,0.000029,0.000000,0.000000,-0.000029,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
